In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Unins

In [ ]:
import numpy as np
import cv2
import open3d as o3d
from tqdm import tqdm
import os
from typing import Tuple, Optional

In [ ]:
# depth value를 z value로 mapping
def map_depth_map_to_point_clouds(depth_map: np.ndarray, step_size: int = 1) -> np.ndarray:
    H, W = depth_map.shape[:2]
    yy, xx = np.meshgrid(range(W), range(H))

    vertices = np.zeros((H, W, 3))
    vertices[..., 1] = xx * step_size
    vertices[..., 0] = yy * step_size
    vertices[..., 2] = depth_map

    return vertices

# 이미지 이동 함수들
def move_left(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((0,0),(0,1)), 'constant', constant_values=0)[:, 1:]

def move_top_right(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((0,1),(1,0)), 'constant', constant_values=0)[1:, :-1]

def move_top(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((1,0), (0,0)), 'constant', constant_values=0)[:-1, :]

def move_top_left(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((1,0), (1,0)), 'constant', constant_values=0)[:-1, :-1]

def move_right(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((0,0), (1,0)), 'constant', constant_values=0)[:, :-1]

def move_bottom(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((0,1), (0,0)), 'constant', constant_values=0)[1:, :]

def move_bottom_right(mask: np.ndarray) -> np.ndarray:
    return np.pad(mask, ((0,1), (0,1)), 'constant', constant_values=0)[1:, 1:]

# image pixel 하나를 face 하나로 정의
def construct_facets_from(mask: np.ndarray) -> np.ndarray:
    idx = np.zeros_like(mask, dtype=int)
    idx[mask] = np.arange(np.sum(mask))

    facet_move_top_mask = move_top(mask)
    facet_move_left_mask = move_left(mask)
    facet_move_top_left_mask = move_top_left(mask)
    facet_top_left_mask = np.logical_and.reduce((facet_move_top_mask, facet_move_left_mask, facet_move_top_left_mask, mask))

    facet_top_right_mask = move_right(facet_top_left_mask)
    facet_bottom_left_mask = move_bottom(facet_top_left_mask)
    facet_bottom_right_mask = move_bottom_right(facet_top_left_mask)

    return np.stack((
        4 * np.ones(np.sum(facet_top_left_mask)),
        idx[facet_top_left_mask],
        idx[facet_bottom_left_mask],
        idx[facet_bottom_right_mask],
        idx[facet_top_right_mask]
    ), axis=-1).astype(int)

# depthmap npy 파일을 읽어 vertex, face 를 return
def get_mesh_from_depth(outname: str, scale: Optional[float] = None) -> Tuple[np.ndarray, np.ndarray]:
    # depth_map_path = os.path.join("./depth_to_textured_mesh", f"{outname}_depth_fp32.npy")
    depth_map_path = os.path.join("/content/drive/MyDrive/KU/AIKU/24-겨울/marigold/depth_to_textured_mesh", f"{outname}.npy")
    depth_map = np.load(depth_map_path)

    if scale is None:
        scale = np.sqrt(depth_map.shape[0] * depth_map.shape[1])

    vertices = map_depth_map_to_point_clouds((1 - depth_map) * scale)
    facets = construct_facets_from(np.ones(depth_map.shape, dtype=bool))

    faces = []
    with tqdm(facets, desc="[Info] Constructing triangular faces") as pbar:
        for face in pbar:
            _, v1, v2, v3, v4 = face
            faces.append([3, v1, v2, v3])
            faces.append([3, v1, v3, v4])

    faces = np.array(faces)
    return vertices, faces

# Textured mesh 생성
def make_textured_mesh(outname: str, scale: Optional[float] = None) -> o3d.geometry.TriangleMesh:
    # texture_path = os.path.join("./depth_to_textured_mesh", f"{outname}.jpeg")
    texture_path = os.path.join("/content/drive/MyDrive/KU/AIKU/24-겨울/marigold/depth_to_textured_mesh", f"{outname}.jpeg")
    textures = cv2.imread(texture_path)
    textures = cv2.cvtColor(textures, cv2.COLOR_BGR2RGB)
    textures = textures / 255.0

    vertices, faces = get_mesh_from_depth(outname, scale)

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices.reshape(-1, 3))
    mesh.triangles = o3d.utility.Vector3iVector(faces[:, 1:])
    mesh.vertex_colors = o3d.utility.Vector3dVector(textures.reshape(-1, 3))

    output_path = os.path.join("./depth_to_textured_mesh/test", f"{outname}_textured_mesh.ply")
    o3d.io.write_triangle_mesh(output_path, mesh)

    return mesh

In [ ]:
cd /content/drive/MyDrive/KU/AIKU/24-겨울/marigold

/content/drive/MyDrive/KU/AIKU/24-겨울/marigold


In [ ]:
def main():
    outname = "output1_pred"  # 변환할 파일 이름 (확장자 제외)
    scale = None  # 필요하면 스케일 조정 가능

    # 필요한 디렉토리 생성
    output_dir = "./depth_to_textured_mesh/test"
    os.makedirs(output_dir, exist_ok=True)

    try:
        print("[Info] Generating 3D mesh from depth map...")
        mesh = make_textured_mesh(outname, scale)
        print(f"[Success] Mesh saved at {output_dir}/{outname}_textured_mesh.ply")

        # Open3D를 사용하여 생성된 Mesh를 시각화
        o3d.visualization.draw_geometries([mesh])

    except Exception as e:
        print(f"[Error] {e}")

if __name__ == "__main__":
    main()

[Info] Generating 3D mesh from depth map...


[Info] Constructing triangular faces: 100%|██████████| 64770/64770 [00:00<00:00, 210683.07it/s]


[Success] Mesh saved at ./depth_to_textured_mesh/test/output1_pred_textured_mesh.ply
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
# import open3d as o3d

# # PLY 파일 경로
# ply_file = "./depth_to_textured_mesh/test/example_3_pred_textured_mesh.ply"

# # PLY 파일 읽기
# mesh = o3d.io.read_triangle_mesh(ply_file)

# # mesh가 정상적으로 로드되었는지 확인
# if not mesh.is_empty():
#     print("Mesh loaded successfully!")

#     # 메시 시각화
#     o3d.visualization.draw_geometries([mesh])
# else:
#     print("Failed to load mesh.")

Mesh loaded successfully!
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np

# PLY 파일 경로
ply_file = "/content/drive/MyDrive/KU/AIKU/24-겨울/output_folders/depth_to_textured_mesh/test/example_3_pred_textured_mesh.ply"

# PLY 파일 읽기
mesh = o3d.io.read_triangle_mesh(ply_file)

# Mesh가 정상적으로 로드되었는지 확인
if not mesh.is_empty():
    print("Mesh loaded successfully!")

    # Open3D를 사용하여 이미지를 렌더링
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)  # 화면에 창을 띄우지 않음
    vis.add_geometry(mesh)

    # 이미지를 렌더링하여 numpy 배열로 저장
    vis.poll_events()
    vis.update_renderer()
    image = vis.capture_screen_float_buffer(False)

    # 이미지를 Matplotlib로 출력
    plt.imshow(np.asarray(image))
    plt.axis('off')  # 축을 없애기
    plt.show()

    vis.destroy_window()  # 렌더링 종료
else:
    print("Failed to load mesh.")